In [71]:
import sys
import csv
import os.path
import re
from collections import defaultdict

In [72]:
def loadSolarWinds(fp):
    """
    load the SolarWinds dump, keyed by IP address
    """
    solarD = {}
    csvR = csv.DictReader(fp)
    for ii,record in enumerate(csvR):
        m = re.match('[A-Z]+_(?P<group>R|VR)',record['Node'])
        if m == None: continue
        record['group'] = m.groupdict()['group']
        solarD[record['IP Address']] = record
    return solarD


In [73]:
def loadStealthWatch(fp):
    """
    load the distinct IP addresses from the StealthWatch dump
    """
    stealthIPs = set()
    csvR = csv.reader(fp)
    for ii,record in enumerate(csvR):
        if ii == 0: continue
        stealthIPs.add(record[0])
    return stealthIPs

In [74]:
def load(solarFname, stealthFname):
    stealthIPs = set()
    with open(stealthFname,'r') as fp:
        stealthIPs = loadStealthWatch(fp)

    solarD = {}
    with open(solarFname,'r') as fp:
        solarD = loadSolarWinds(fp)

    return solarD, stealthIPs

In [75]:
def find_missing(solarD, stealthIPs):
    """
    find the IPs that in solarWinds, but not stealthWatch
    """
    solarIPs = set(solarD.keys())
    missing = solarIPs - stealthIPs
    return missing


In [76]:
def report(solarD,missing):
    reportL = []
    groupD = defaultdict(list)
    for ip in missing:
        solar = solarD[ip]
        groupD[solar['group']].append(solar)
    for group, groupL in sorted(groupD.items()):
        reportL.append( '{0} is missing {1} entries in StealthWatch'.format(group,len(groupL)))
        reportL.append('\t' + '\n\t'.join(
                ['{0}\t{1}'.format(
                        record['Node'],record['IP Address']
                    ) for record in sorted(groupL,key=lambda rec:rec['Node'])]
            ))
    return reportL


In [77]:
solarFname = '/Users/gardner/Downloads/Solar.csv'
stealthFname = '/Users/gardner/Downloads/RCStealthWatchExport.csv'

solarD, stealthIPs = load(solarFname, stealthFname)
print ('%d distinct IPs in StealthWatch' % len(stealthIPs))
print ('%d SolarWinds records' % len(solarD.keys()))

missing = find_missing(solarD,stealthIPs)
print( '%d IPs missing from StealthWatch' % len(missing))

reportL = report(solarD,missing)
print('\n'.join(reportL))


153 distinct IPs in StealthWatch
182 SolarWinds records
118 IPs missing from StealthWatch
R is missing 40 entries in StealthWatch
	AAA_R02.man.cox.com	10.131.16.10
	ALOH_R01.man.cox.com	10.131.16.57
	BCAA_R02.man.cox.com	10.105.64.146
	BELA_R01.man.cox.com	10.140.0.1
	CADE_R02.man.cox.com	10.105.64.186
	CADE_RSR01.man.cox.com	10.135.8.1
	CLEV_R01.man.cox.com	10.131.19.201
	DADE_R01.man.cox.com	10.105.64.233
	DALA_R02.man.cox.com	10.105.65.10
	DETA_R02.man.cox.com	10.105.65.18
	EDMO_R01.man.cox.com	10.105.67.65
	FAAO_R02.man.cox.com	10.105.65.50
	GCAA_R02.man.cox.com	10.105.65.74
	HAA_R01.man.cox.com	10.131.1.154
	KEYA_R01.man.cox.com	10.105.65.145
	KLOC_R01.man.cox.com	10.105.67.57
	KLOD_R01.man.cox.com	10.105.67.89
	LATL_R01.man.cox.com	10.131.16.233
	LHST_R01.man.cox.com	10.131.17.209
	LPHX_R01.man.cox.com	10.131.19.209
	MAA_R02.man.cox.com	10.105.65.186
	NAA_R01.man.cox.com	10.105.66.17
	NADE_R02.man.cox.com	10.105.66.26
	NASH_R02.man.cox.com	10.105.66.42
	NGC_R01.man.cox.com	10.131